In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import sklearn

In [3]:
lookup_table=pd.read_csv('/home/naveen/datasets/Facial Keypoints/facial-keypoints-detection/IdLookupTable.csv')

In [4]:
lookup_table.head()

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN


In [5]:
data_train=pd.read_csv('/home/naveen/datasets/Facial Keypoints/facial-keypoints-detection/training.csv')

In [6]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7049 entries, 0 to 7048
Data columns (total 31 columns):
left_eye_center_x            7039 non-null float64
left_eye_center_y            7039 non-null float64
right_eye_center_x           7036 non-null float64
right_eye_center_y           7036 non-null float64
left_eye_inner_corner_x      2271 non-null float64
left_eye_inner_corner_y      2271 non-null float64
left_eye_outer_corner_x      2267 non-null float64
left_eye_outer_corner_y      2267 non-null float64
right_eye_inner_corner_x     2268 non-null float64
right_eye_inner_corner_y     2268 non-null float64
right_eye_outer_corner_x     2268 non-null float64
right_eye_outer_corner_y     2268 non-null float64
left_eyebrow_inner_end_x     2270 non-null float64
left_eyebrow_inner_end_y     2270 non-null float64
left_eyebrow_outer_end_x     2225 non-null float64
left_eyebrow_outer_end_y     2225 non-null float64
right_eyebrow_inner_end_x    2270 non-null float64
right_eyebrow_inner_end_y 

In [8]:
data_train.shape

(7049, 31)

In [16]:
#7049 images with corresponding keypoint locations
data_train['Image'].shape

(7049,)

In [18]:
#plt.imshow(data_train['Image'])

In [19]:
data_test=pd.read_csv('/home/naveen/datasets/Facial Keypoints/facial-keypoints-detection/test.csv')

In [20]:
data_test

,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...
5,6,177 177 177 171 142 115 97 84 89 90 88 82 63 5...
6,7,77 55 44 56 58 61 67 66 65 65 65 73 72 78 86 9...
7,8,156 160 162 166 150 114 97 105 125 127 119 121...
8,9,230 230 231 231 231 231 231 231 231 229 214 20...
9,10,132 129 126 128 146 163 170 169 143 101 71 55 ...
